In [139]:
from bs4 import BeautifulSoup
import requests
root_url = 'https://www.immoweb.be/en'
#cookie_url = root_url + '/cookie'
#cookies = requests.get(cookie_url).cookies
#print(cookies)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}
### to get to the page-urls (seemingly always with 30 properties per pages) for list of 10000  houses (only), 
### only in BE and excluding life annuity sales (hence 2 filters applied)
### sorted by newest first, language set to English
pages = range(332) # assuming 333 pages if range is set to 332, assumption could be checked with last 'li' tag and class = "pagination__item" on the first page url (TBC)
### !!!! improvement probabaly would be to use the <next page> button instead of recalling each time with new request.  As the list can change continiously, we might end up with dupplicates now.
listing_url_pages = []
listing_url_properties = []
for i in pages:
    page = i+1
    page_url=f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page={page}&orderBy=relevance&orderBy=newest"
    listing_url_pages.append(page_url)
print(listing_url_pages)
print(f"number of page-urls retrieved : {len(listing_url_pages)}")
Type_of_property = page_url.split('/')[5]
print(f"Type_of_property retrieved : {Type_of_property}") # with this search url, we are only looking for properties od type "house"

['https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance&orderBy=newest', 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=2&orderBy=relevance&orderBy=newest', 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=3&orderBy=relevance&orderBy=newest', 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=4&orderBy=relevance&orderBy=newest', 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=5&orderBy=relevance&orderBy=newest', 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=6&orderBy=relevance&orderBy=newest', 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=7&orderBy=relevance&orderBy=newest', 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=8&or

In [140]:
def list_propertie_urls(page_url):
    """function returns a listing of property-urls 
    from an url to be provided as parameter (url representing of a listing-page of immoweb.be/en """
    list_1_page = []
    req_1_page = requests.get(page_url, headers=headers)
    page_soup = BeautifulSoup(req_1_page.text, 'html.parser')
    listings = page_soup.find_all('article', class_='card--result')
    for listing in listings:
        list_1_page.append(listing.find('a', class_='card__title-link', href=True)['href'])
    return (list_1_page)   

#list_propertie_urls('https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance&orderBy=newestl)')
#list_propertie_url(listing_url_pages[-1]) # test when on last page

In [141]:
# looping over a list of pages and retrieve all urls for the individual properties shown on those pages
for i in listing_url_pages:
    listing_url_properties += (list_propertie_urls(i))
print(listing_url_properties)
print(f"nbr of individual properties (urls) retrieved : {len(listing_url_properties)}")

['https://www.immoweb.be/en/classified/house/for-sale/wezembeek-oppem/1970/20238091', 'https://www.immoweb.be/en/classified/house/for-sale/maldegem/9990/20238089', 'https://www.immoweb.be/en/classified/house/for-sale/maurage/7110/20238088', 'https://www.immoweb.be/en/classified/house/for-sale/geraardsbergen/9500/20238086', 'https://www.immoweb.be/en/classified/house/for-sale/putte/2580/20238085', 'https://www.immoweb.be/en/classified/house/for-sale/flemalle/4400/20147096', 'https://www.immoweb.be/en/classified/house/for-sale/destelbergen/9070/20238082', 'https://www.immoweb.be/en/classified/house/for-sale/waregem/8790/20238080', 'https://www.immoweb.be/en/classified/house/for-sale/izegem/8870/20238078', 'https://www.immoweb.be/en/classified/villa/for-sale/aywaille/4920/20069534', 'https://www.immoweb.be/en/classified/house/for-sale/herve/4650/20238075', 'https://www.immoweb.be/en/classified/house/for-sale/brugge-assebroek/8310/20238072', 'https://www.immoweb.be/en/classified/house/for-

In [142]:
def list_properties_detail(property_url):
    """function returns a dictionary of attributes of 1 specific property 
    from an property url to be provided as parameter """
    req_1_house = requests.get(property_url, headers=headers)
    if req_1_house.status_code == 200:         # check if still valid url !!!!!    
        soup = BeautifulSoup(req_1_house.text, 'html.parser')
        listing_data = {}
        ### ID ###
        for x in soup.find('div', attrs={"class": "classified__header--immoweb-code"}):
            listing_data["Property_ID"] = x.text.split(":")[-1].strip()
        ### PRICE ###
        price_container = soup.find('p', attrs={"class": "classified__price"})
        sr_only_span = price_container.find('span', attrs={"class": "sr-only"})
        if sr_only_span:
            listing_data["Price"] = price_container.find('span', attrs={"class": "sr-only"}).text  
        ### MISC ###
        rows = soup.find_all('tr', attrs={"class" :'classified-table__row'})
        for row in rows:
        #print("row : ",row)
            if row.find('th', attrs={"class" :'classified-table__header'}):
                header = row.find('th', attrs={"class" :'classified-table__header'}).contents
            key_field =((header[0]).strip()) 
            #print(key_field)  
            if row.find('td', attrs={"class" :'classified-table__data'}):  
                data = row.find('td', attrs={"class" :'classified-table__data'}).contents
            value_field = ((data[0]).strip())
            #print(value_field) 
            relevant_keys = ["Living area","Terrace surface","Swimming pool", "Number of frontages", "Bedrooms", "How many fireplaces?", "Kitchen type", "Garden surface", "Swimming pool", "Type of building", "Furnished"]   
            if key_field in relevant_keys:
                listing_data[key_field]= value_field
        ### MISC2 ###
        parts = property_url.split('/')
        listing_data['Postal_Code']=parts[8]
        listing_data['Locality_Name']=parts[7]  
        listing_data['extra']=parts[6] 
        listing_data['Subtype_of_property']=parts[5] 
        listing_data['Type_of_property']=Type_of_property     
        return listing_data

#list_properties_detail('https://www.immoweb.be/en/classified/villa/for-sale/braine-lalleud/1421/20237028') 

In [143]:
listing_properties_detail = []
for i in listing_url_properties:  
    print(i)
    listing_properties_detail.append(list_properties_detail(i))

        

https://www.immoweb.be/en/classified/house/for-sale/wezembeek-oppem/1970/20238091
https://www.immoweb.be/en/classified/house/for-sale/maldegem/9990/20238089
https://www.immoweb.be/en/classified/house/for-sale/maurage/7110/20238088
https://www.immoweb.be/en/classified/house/for-sale/geraardsbergen/9500/20238086
https://www.immoweb.be/en/classified/house/for-sale/putte/2580/20238085
https://www.immoweb.be/en/classified/house/for-sale/flemalle/4400/20147096
https://www.immoweb.be/en/classified/house/for-sale/destelbergen/9070/20238082
https://www.immoweb.be/en/classified/house/for-sale/waregem/8790/20238080
https://www.immoweb.be/en/classified/house/for-sale/izegem/8870/20238078
https://www.immoweb.be/en/classified/villa/for-sale/aywaille/4920/20069534
https://www.immoweb.be/en/classified/house/for-sale/herve/4650/20238075
https://www.immoweb.be/en/classified/house/for-sale/brugge-assebroek/8310/20238072
https://www.immoweb.be/en/classified/house/for-sale/bertrix/6880/20238071
https://www

In [153]:
### save data with json
import json
def save(file_to_be_save):
    with open('listings.json', 'w') as f:
        json.dump(listing_properties_detail, f)

save(listing_properties_detail)

In [145]:
### save data with pd to csv format
import pandas as pd
from datetime import datetime
df = pd.DataFrame.from_dict(listing_properties_detail)
print(f"shape of the dataframe is {df.shape} with: (1)the number of properties -in rows, (2) the number of attributes -in columns")

AttributeError: 'NoneType' object has no attribute 'keys'

In [138]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
filename = rf'C:\repos\immo-eliza-scraping\data\raw\df_{timestamp}.csv'
df.to_csv(filename, index=False)
